In [1]:
from typing import Union

from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as                           EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException, WebDriverException, NoSuchElementException, ElementClickInterceptedException

from bs4 import BeautifulSoup
import random
import time
import csv
import re

### Some variables

In [2]:
FIREFOX_DRIVER = "/home/nick/apps/geckodriver"
PROBLEMS_FILE = "linkuri_probleme.txt"
FREE_PROBLEMS_FILE = "linkuri_free_probleme.txt"

In [3]:
website_url = "https://leetcode.com/problemset/"
service = FirefoxService(FIREFOX_DRIVER)  # Exemplu: /usr/local/bin/geckodriver
fremium_problems = []
user_timeout = 3

### Driver options:

In [4]:
def dump_page(bs: BeautifulSoup, filename: str="file.html") -> None:
    with open(filename, "w") as f1:
        f1.write(bs.prettify())

In [5]:
def dump_fremium_problems():
    global fremium_problems
    with open(FREE_PROBLEMS_FILE, "r") as g:
        fremium_problems = g.readlines()
def dump_all_problems():
    global problemset_links
    with open(PROBLEMS_FILE, "r") as g:
        problemset_links = g.readlines()

## SKIP to finding Discussion

In [6]:
dump_fremium_problems()
# dump_all_problems()
# print(len(fremium_problems))
# print(fremium_problems)

# Webdriver's code:

In [8]:
firefox_options = Options()
# firefox_options.add_argument("--headless")

# Inițializează driverul pentru Firefox
driver = webdriver.Firefox(service=service, options=firefox_options)

In [17]:
# Accesează pagina dorită
driver.get(website_url)
driver.set_page_load_timeout(3)
print("Am accesat pagina cu Firefox")

Am accesat pagina cu Firefox


#### Sari 2 celule daca nu vrei sa recitesti toate problemele

In [14]:
f = open(PROBLEMS_FILE, "a")

# Asta se va asigura ca fisierul exista si este gol
# with open(PROBLEMS_FILE, 'w'):
#     pass
wait = WebDriverWait(driver, user_timeout)
problemset_links = []

Modific codul asta doar dupa ce voi strange mai mult de o pagina de probleme

In [15]:

# Iterează prin numarul de paginii de pe leetcode practice
i = 1

for problems_page in range(70):
    try:
        anchors = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.TAG_NAME, "a")))
        anchors = [a for a in anchors if re.match(r"^\d+\.", a.text)] # Aici am pastrat doar ancorele cu problemset-uri efectiv
        
        # Itereaza prin linkurile de pe pagina respectiva
        for anchor in anchors:
            try:
                link_text = anchor.text
                link_url = anchor.get_attribute("href")
            
                problemset_links.append(link_url)
                f.write(f"{link_url}\n")
            except TimeoutException as e:
                print(f"TimeoutException la pagina {i}")
                time.sleep(random.randint(3, 7))
    except StaleElementReferenceException as e:
        print(f"StaleElementReferenceException la pagina {i}")
        continue

    try:
        next_page = wait.until(
            EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='next']"))
        )
        next_page.click()
        wait.until(EC.staleness_of(anchors[0]))
    except TimeoutException as e:
        print(f"TimeoutException la pagina {i} -> cel mai probabil {i} este ultima pagina")
        break
    i += 1

KeyboardInterrupt: 

TEST

In [ ]:
print(problemset_links[156])
print(len(problemset_links))

#### Mai sari 3 celule sa ajungi la 'Discussions' topic
# Extract fremium problemset
#### Prima mare iteratie va fi printre problemele free

In [ ]:
var = 0
linkuri_problematice = []
def test_is_premium(link: str) -> bool:
    global var, driver
    var += 1
    if var % 40 == 0:
        print(f"i = {var}")
    try:
        driver.get(link)
        subscrieburi = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.TAG_NAME, "a")))
        premium = False
        for sub in subscrieburi:
            if re.match(r"^Subscribe", sub.text):
                premium = True
                break
        driver.back()
        # print(f'i = {var} bine')
        return premium
    except StaleElementReferenceException:
        linkuri_problematice.append(link)
        print(f"i = {var} -> \'StaleElementReferenceException\' la linkul {link}")
        return True
    except TimeoutException:
        linkuri_problematice.append(link)
        print(f"i = {var} -> TimeoutException la linkul {link}")
        time.sleep(random.randint(3, 7)) # Asteptam impotriva bannarii
        return True
    except WebDriverException as ee:
        if "out of memory" in str(ee):
            print(f"i = {var} 'out of memory' occured. Restarting driver")
            driver.quit()
            driver = webdriver.Firefox(service=service, options=firefox_options)
            return test_is_premium(link)
        else:
            return True
            

In [ ]:
fremium_problems = [link for link in problemset_links if not test_is_premium(link)]
# with open(FREE_PROBLEMS_FILE, "w") as file:
#     for fremium in fremium_problems:
#         file.write(fremium + '\n')

In [29]:
driver.quit()

In [30]:
firefox_options = Options()
# firefox_options.add_argument("--headless")

# Inițializează driverul pentru Firefox
driver = webdriver.Firefox(service=service, options=firefox_options)
# idx = random.randint(0, len(fremium_problems))
idx = 10
problem = fremium_problems[idx]
print(problem)
driver.get(problem)
time.sleep(5)

https://leetcode.com/problems/regular-expression-matching



### finding 'Discussion' section

In [31]:
try:
    WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[contains(text(), 'Discussion')]"))
    )
    discussion_element = driver.find_element(By.XPATH, "//*[contains(text(), 'Discussion')]")
    discussion_element.click()  # Dă click pe element
    print("Click pe elementul 'Discussion' efectuat cu succes!")
except Exception as e:
    print(f"Eroare: {e}")

Click pe elementul 'Discussion' efectuat cu succes!


### finding 'Topics' section

In [21]:
def topics() -> Union[str, None]:
    try:
        WebDriverWait(driver, 2).until(
            EC.presence_of_all_elements_located((By.XPATH, "//a[contains(@href, '/tag/')]"))
        )
        tags = driver.find_elements(By.XPATH, "//a[contains(@href, '/tag/')]")
        print("Click pe elementul 'Topics' efectuat cu succes!\n")
        for tag_ in tags:
            l = tag_.get_attribute('outerHTML')
            start = l.find(">") + 1
            stop = l.find("<", start + 1)
            print(l[start:stop])
            return l[start:stop]
    except Exception as e_:
        print(f"Eroare: {e_}")
        return None

### finding 'Difficulty' section

In [22]:
def difficulty() -> Union[str, None]:
    try:
        WebDriverWait(driver, 2).until(
            EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, 'text-difficulty')]"))
        )
        tag = driver.find_element(By.XPATH, "//div[contains(@class, 'text-difficulty')]")
        print(tag.text)
        return tag.text
    except Exception as e_:
        print(f"Eroare: {e_}")
        return None

### finding 'Accepted & Submissions & Acceptance-Rate' section

In [23]:
def acc_sub_accrate() -> Union[tuple[str, str, str], tuple[None, None, None]]:
    try:
        WebDriverWait(driver, 2).until(
            EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, 'text-label-1 dark:text-dark-label-1 text-sm font-medium')]"))
        )
        values = driver.find_elements(By.XPATH, "//div[contains(@class, 'text-label-1 dark:text-dark-label-1 text-sm font-medium')]")
        for value in values:
            print(value.text)
        tup = (value.text for value in values)
        return tup
    except Exception as e_:
        print(f"Eroare: {e_}")
        return None, None, None

Teste foarte bune: <br>
problema 876 -> 10 pagini comentarii <br><br>
problema 2407 -> 3 pagini comentarii <br><br>
problema 6 -> 33 pagini comentarii; primul comentari de pe prima pagina are 21 raspunsuri si trebuie aditional apasat butonul 'show more replies' <br><br>
problema 8 -> 58 de pagini(insane); primul comentariu de pe prima pagina are 12 raspunsuri si trebuie apasat butonul de 'show more replies'

In [24]:
consent = False

In [52]:
import time

def go_to_next_comment_page(drv, timeout=2) -> bool:
    try:
        # drv.find_element(By.XPATH, "//button[@aria-label='next']")
        next_button = WebDriverWait(drv, timeout).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='next']"))
            )
        if next_button:
            next_button.click()
        # print("a mers ok")
        return True
    except (NoSuchElementException, TimeoutException):
        print(f"NoSuchElementException / TimeoutException -> cel mai probabil butonul este inactiv, semn ca este ultima pagina de discutii")
        return False
    except ElementClickInterceptedException:
        print("ElemetClickInterceptedException")
        return False

def expand_read_more(parent_div) -> bool:
    try:
        read_more_button = WebDriverWait(parent_div, 1).until(
            EC.element_to_be_clickable((By.XPATH, ".//div[contains(@class, 'text-md flex w-full items-center justify-center text-label-1 dark:text-dark-label-1')]"))
        )
        read_more_button.click()
        expand_read_more(parent_div)
        return True
    except (NoSuchElementException, ElementClickInterceptedException, TimeoutException):
        return False

def has_responses(div) -> bool:
    global consent
    while True:
        try:
            reply = WebDriverWait(div, 2).until(
                EC.element_to_be_clickable((By.XPATH, ".//div[contains(@class, 'flex items-center gap-1 group shrink-0 cursor-pointer transition-colors')]"))
            )
            reply.click()
            time.sleep(0.4)
            return True
        except (NoSuchElementException, TimeoutException):
            # print("TimeoutException in has_responses")
            return False
        except ElementClickInterceptedException:
            # print("ElementClickInterceptedException in has_responses")

            if not consent:
                try:
                    consent_button = WebDriverWait(div, 2).until(
                        EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Consent']"))
                    )
                    print("Am gasit butonul de use data Consent!")
                    consent_button.click()
                    consent = True
                except TimeoutException:
                    print("Butonul de consimțământ nu este disponibil.")
                    return False  # Ieșim din buclă pentru a preveni repetarea
            else:
                # Încearcă să găsești butonul cookie
                try:
                    cookie_button = WebDriverWait(div, 2).until(
                        EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Dismiss privacy and legal settings display']"))
                    )
                    # print("Am gasit butonul de google cookie!")
                    cookie_button.click()
                except TimeoutException:
                    # print("Butonul de cookie nu este disponibil.")
                    return False  # Ieșim din buclă dacă nu mai există cookie popup
            
            
def get_date(div) -> Union[str, None]:
    try:
        date_tag = WebDriverWait(div, 2).until(
            EC.element_to_be_clickable((By.XPATH, ".//span[contains(@class, 'closed')]"))
        )
        return date_tag.text
    except (NoSuchElementException, TimeoutException):
        return None
    

Tot acest bloc de cod va trebui pus in interiorul unui alt bloc care itereaza prin toate problemele de pe leetcode <br>
In exterior raman variabilele *user_timeout*, iar in prima bucla vor fi *global_idx*, *comment_id* si *pag*

**Celula care merge** integral si care arata doar comentariile principale

In [11]:
import io

def pretty_print(pag_: int, user_: str, id_: int, parent_: int, date: Union[None, str], upvotes, file_=None) -> None:
    output =f"page: {pag_:>2} | user: {user_:>27} | id: {id_:>4} | parent: {parent_:>4} | date: {date:>9} | upvotes: {upvotes:>4}"
    if file_:
        if isinstance(file_, io.TextIOWrapper):
            file_.write(output + '\n')
        else:
            return
    else:
        print(output)

pretty_print(5, "alldn", 54, 0, "23 Oct, 2023", "534")
pretty_print(22, "Raveesh_gautam", 241, 0, "15 Jan, 2019", "193")

page:  5 | user:                       alldn | id:   54 | parent:    0 | date: 23 Oct, 2023 | upvotes:  534
page: 22 | user:              Raveesh_gautam | id:  241 | parent:    0 | date: 15 Jan, 2019 | upvotes:  193


In [13]:
csv_1 = open("problemset.csv", mode="w", newline="")
csv_2 = open("discussion.csv", mode="w", newline="")

writer_1 = csv.writer(csv_1)
writer_2 = csv.writer(csv_2)

writer_1.writerow(["Id", "Problem Name", "Description", "Accepted", "Submissions", "Acceptance Rate"])
writer_2.writerow(["Problem Name", "Id", "Parent_Id", "Text", "Date", "Up Votes"])

46

In [44]:
driver.quit()
firefox_options = Options()
# firefox_options.add_argument("--headless")

# Inițializează driverul pentru Firefox
driver = webdriver.Firefox(service=service, options=firefox_options)
# idx = random.randint(0, len(fremium_problems))
idx = 10
problem = fremium_problems[idx]
print(problem)
driver.get(problem)

https://leetcode.com/problems/regular-expression-matching



In [58]:
pag = 1
comment_id = 0

def seek_for_responses():
    global comment_id, pag
    while True:
        try:
            # Aici gasesc lista de comentarii
            com_ancs = WebDriverWait(driver, user_timeout).until(
                EC.presence_of_all_elements_located((By.XPATH, "/*//a[starts-with(@href, '/u/') and normalize-space(text()) != '']"))
            )
            
            # afiseaza userii de pe acelasi nivel
            for com_anc in com_ancs:
                comment_id += 1
                try:
                    # class = flex w-full flex-col py-3
                    sixth_parent_div = com_anc.find_element(By.XPATH, "ancestor::div[6]")
                    
                    # Gaseste parintele lui `sixth_parent_div`
                    # print(parent_div.get_attribute('class')
                    

                    # class = mt-2 flex w-full flex-col text-label-2 dark:text-dark-label-2
                    second_child_div = sixth_parent_div.find_element(By.XPATH, "div[2]")
                    # class = FN9Jv
                    fourth_descendant_div = second_child_div.find_element(By.XPATH, "./descendant::div[4]")
                    # COMMENT TEXT
                    split_comment_texts = fourth_descendant_div.find_elements(By.XPATH, ".//p")
                    comment_text = ""
                    for c_text in split_comment_texts:
                        comment_text += c_text.text
                    
                    # DATE
                    date_tag = sixth_parent_div.find_element(By.XPATH, ".//span")
                    date = date_tag.text
                    
                    # UPVOTES
                    upvotes_div = second_child_div.find_element(By.XPATH, ".//div[@class='text-xs cursor-pointer text-label-3 dark:text-dark-label-3 hover:text-label-2 dark:hover:text-dark-label-2']")
                    upvotes = upvotes_div.text
                    
                    pretty_print(pag, com_anc.text, comment_id, 0, date, upvotes)
                    writer_2.writerow(["regular-expression-matching", comment_id, 0, comment_text, date, upvotes])
                    
                    if expand_read_more(sixth_parent_div):
                        print(f"\tHAS 'READ MORE' BUTTON")
                    if has_responses(sixth_parent_div):
                        parent_id = comment_id
                        comment_id += 1
                        print("\tHAS RESPONSES!")
                        # Selecteaza toti fratii parintelui lui `sixth_parent_div` care au clasa `flex flex-col`
                        # print(sixth_parent_div.get_attribute("class"))
                        
                        # px-1 transition-[background] duration-500
                        parent_div = sixth_parent_div.find_element(By.XPATH, "..")
                        # print(f"parent_div -> {parent_div.get_attribute('class')}")

                        a = input(len(parent_div.find_elements(By.XPATH, "./*")))
                        print()
                        # flex flex-col
                        responses_block = WebDriverWait(driver, 3).until(
                            EC.presence_of_element_located((By.XPATH, "//parent_div_xpath/div[1]"))
                        )
                        # print(f"responses_block -> {responses_block.get_attribute('class')}")
                        
                        responses = responses_block.find_elements(By.XPATH, ".//div[@class='flex flex-col']")
                        print(f"len(responses) -> {len(responses)}")

                        for response in responses:
                            # USERNAME
                            user_anchor_tag = response.find_element(By.XPATH, ".//a[contains(@href, '/u/') and normalize-space(text()) != '']")
                            st = len("https://leetcode.com/u/")
                            username = user_anchor_tag.get_attribute("href")[st:-2]
                            
                            # DATE
                            span_tag = response.find_element(By.XPATH, ".//span")
                            date = span_tag.text
                            
                            # UP-VOTES
                            up_votes_tag = response.find_element(".//div[class='text-xs cursor-pointer text-label-3 dark:text-dark-label-3 hover:text-label-2 dark:hover:text-dark-label-2']")
                            up_votes = up_votes_tag.text
                            
                            # TEXT
                            wrapper = response.find_element(".//div[class='FN9Jv]")
                            text_tag = wrapper.find_element(".//p")
                            
                            print(f'user: {username} | p_ID: {parent_id} | ID: {comment_id} | votes: {up_votes} | date: {date}')
                        
                except StaleElementReferenceException:
                    break
                    
            else:
                if go_to_next_comment_page(driver):
                    pag += 1
                else:
                    break
                
        except StaleElementReferenceException:
            print(f"Page {pag} elements went stale, retrying on this page...")
            continue

        # go_to_next_comment_page(driver)
        # pag += 1
            
seek_for_responses()

page:  1 | user:               shawnForsythe | id:    1 | parent:    0 | date: Oct 29, 2013 | upvotes:  600
	HAS RESPONSES!



TimeoutException: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16


In [ ]:
for pb in fremium_problems:
    driver.get(pb)
    seek_for_responses()

Buton de comentarii: <br>
viewBox="0 0 24 24"